In [36]:
import time
import boto3
import json
import numpy as np
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

# Configuring the built-in Image Classification algorithm

## Configure Hyperparameters

In [37]:
# Number of output classes
num_classes = 2

# Number of training samples in the training set
num_training_samples = 194266

# Number of layers for the underlying neural network
num_layers = 18

# Batch size for training
mini_batch_size =  128

# Input image shape for the training data
image_shape = '3,50,50'

# Augmentation type
augmentation_type = 'crop_color_transform'

# Number of epochs
epochs = 5

# Learning rate
learning_rate = 0.01

# Enable transfer learning
use_pretrained_model = 1

## Create a unique job name 

In [38]:
job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

In [39]:
bucket = 'nitinproject1'
input_prefix = 'breast-cancer-detection/input/recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

In [40]:
output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

In [41]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role and the training image URI for Image Classification

In [42]:
role = get_execution_role()
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

## Configure train timeout

In [43]:
train_timeout = 360000

## Putting it all together

In [44]:
training_params = {
    'TrainingJobName': job_name,
    'AlgorithmSpecification': {
        'TrainingImage': training_image,
        'TrainingInputMode': 'Pipe'
    },
    'RoleArn': role,
    'ResourceConfig': {
        'InstanceCount': instance_count,
        'InstanceType': instance_type,
        'VolumeSizeInGB': volume_size_gb
    },
    'InputDataConfig': [
        {
            'ChannelName': 'train',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': input_train,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'ContentType': 'application/x-recordio',
            'CompressionType': 'None'
        },
        {
            'ChannelName': 'validation',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': input_test,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'ContentType': 'application/x-recordio',
            'CompressionType': 'None'
        }
    ],
    'OutputDataConfig': {
        'S3OutputPath': output_path
    },
    'HyperParameters': {
        'num_classes': str(num_classes),
        'num_training_samples': str(num_training_samples),
        'num_layers': str(num_layers),
        'mini_batch_size': str(mini_batch_size),
        'image_shape': image_shape,
        'augmentation_type': augmentation_type,
        'epochs': str(epochs),
        'learning_rate': str(learning_rate),
        'use_pretrained_model': str(use_pretrained_model)
    },
    'StoppingCondition': {
        'MaxRuntimeInSeconds': train_timeout
    }
}

# Creating a training job

In [10]:
sagemaker_client = boto3.client(service_name='sagemaker')
sagemaker_client.create_training_job(**training_params)

# Confirm the training job has started
status = sagemaker_client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

try:
    # Wait for the job to finish and report the ending status
    sagemaker_client.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    training_info = sagemaker_client.describe_training_job(TrainingJobName=job_name)
    status = training_info['TrainingJobStatus']
    print('Training job ended with status: ' + status)
except:
    print('Training failed to start')
    message = sagemaker_client.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job current status: InProgress
Training job ended with status: Completed


## Obtaining the model S3 URI

In [11]:
model_uri = sagemaker_client.describe_training_job(TrainingJobName=job_name)['ModelArtifacts']['S3ModelArtifacts']

In [12]:
model_uri

's3://nitinproject1/breast-cancer-detection/output/breast-cancer-detection-2020-05-07-18-39-49/output/model.tar.gz'

# Creating a tuning job

## Defining tuning configuration

In [13]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [
        {
          "Name": "optimizer",
          "Values": ["sgd", "adam"]
        }          
      ],
      "ContinuousParameterRanges": [
        {
          "Name": "learning_rate",
          "MinValue": "0.001",
          "MaxValue": "1.0"
        }
      ],
      "IntegerParameterRanges": [
        {
          "Name": "mini_batch_size",
          "MinValue": "64",
          "MaxValue": "128"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 2,
      "MaxParallelTrainingJobs": 2
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:accuracy",
      "Type": "Maximize"
    }
}

## Defining training parameters

In [14]:
training_params = {
    'AlgorithmSpecification': {
        'TrainingImage': training_image,
        'TrainingInputMode': 'Pipe'
    },
    'RoleArn': role,
    'ResourceConfig': {
        'InstanceCount': instance_count,
        'InstanceType': instance_type,
        'VolumeSizeInGB': volume_size_gb
    },
    'InputDataConfig': [
        {
            'ChannelName': 'train',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': input_train,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'ContentType': 'application/x-recordio',
            'CompressionType': 'None'
        },
        {
            'ChannelName': 'validation',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': input_test,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'ContentType': 'application/x-recordio',
            'CompressionType': 'None'
        }
    ],
    'OutputDataConfig': {
        'S3OutputPath': output_path
    },
    'StaticHyperParameters': {
        'image_shape': image_shape,
        'num_layers': str(num_layers),
        'num_training_samples': str(num_training_samples),
        'num_classes': str(num_classes),
        'epochs': str(epochs),
        'use_pretrained_model': str(use_pretrained_model)
    },
    'StoppingCondition': {
        'MaxRuntimeInSeconds': 360000
    }
}

## Create a unique job name

In [15]:
job_name_prefix = 'bcd-tuning'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Launching the tuning job

In [ ]:
sagemaker_client = boto3.client(service_name='sagemaker')
sagemaker_client.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = job_name,
                                                   HyperParameterTuningJobConfig = tuning_job_config,
                                                   TrainingJobDefinition = training_params)

# Deploying the best model found by the tuning job

## Get the execution role and the hosting image URI for Image Classification

In [45]:
role = get_execution_role()
hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

## Create a unique model name

In [46]:
model_name_prefix = 'bcd-image-classification-low-level'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = model_name_prefix + timestamp

## Creating a model from training output (model artifacts)

In [47]:
sagemaker_client = boto3.client(service_name='sagemaker')

model_artifacts_s3_path = 's3://nitinproject1/breast-cancer-detection/output/breast-cancer-detection-2020-05-07-18-39-49/output/model.tar.gz'

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': hosting_image,
        'ModelDataUrl': model_artifacts_s3_path,
    }
)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-1:263771021365:model/bcd-image-classification-low-level-2020-05-10-20-58-31


## Create a unique endpoint configuration name

In [48]:
endpoint_config_name_prefix = 'breast-cancer-detection-epc'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = endpoint_config_name_prefix + timestamp

## Configure hosting instances

In [49]:
instance_count = 1
instance_type = 'ml.m4.xlarge'

## Create an endpoint configuration

In [50]:
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [{
        'VariantName': 'AllTraffic',
        'InstanceType': instance_type,
        'InitialInstanceCount': instance_count,
        'ModelName': model_name
    }]
)

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration name: breast-cancer-detection-epc-2020-05-10-20-58-39
Endpoint configuration arn:  arn:aws:sagemaker:us-east-1:263771021365:endpoint-config/breast-cancer-detection-epc-2020-05-10-20-58-39


## Create a unique endpoint name

In [51]:
endpoint_name_prefix = 'breast-cancer-detection-ep'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = endpoint_name_prefix + timestamp

In [52]:
endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
print('Endpoint name: {}'.format(endpoint_name))
print('Endpoint arn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: breast-cancer-detection-ep-2020-05-10-20-58-48
Endpoint arn = arn:aws:sagemaker:us-east-1:263771021365:endpoint/breast-cancer-detection-ep-2020-05-10-20-58-48


In [53]:
# Get the status of the endpoint
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))

# Wait until the status has changed
sagemaker_client.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)

# Print the status of the endpoint
endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

EndpointStatus = Creating
Endpoint creation ended with EndpointStatus = InService


# Testing the deployed model

In [54]:
sagemaker_runtime_client = boto3.Session().client(service_name='runtime.sagemaker')

In [55]:
def predict_breast_cancer(image_path):
    with open(image_path, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
    response = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/x-image', 
        Body=payload
    )
    result = response['Body'].read()
    result = json.loads(result)
    print('Probabilities for all classes: ', result)
    predicted_class = np.argmax(result)
    if predicted_class == 0:
        print('Breast cancer not detected')
    else:
        print('Breast cancer detected')

In [56]:
predict_breast_cancer('images/0/8975_idx5_x2851_y1201_class0.png')

Probabilities for all classes:  [0.5578531622886658, 0.44214683771133423]
Breast cancer not detected


In [57]:
predict_breast_cancer('images/1/10253_idx5_x551_y651_class1.png')

Probabilities for all classes:  [0.5094565749168396, 0.490543395280838]
Breast cancer not detected


In [58]:
predict_breast_cancer('samples/fake_images-main.png')

predict_breast_cancer('samples/fake_images-1.png')

predict_breast_cancer('samples/fake_images-2.png')

predict_breast_cancer('samples/fake_images-3.png')

predict_breast_cancer('samples/fake_images-4.png')

predict_breast_cancer('samples/fake_images-5.png')

predict_breast_cancer('samples/fake_images-6.png')

predict_breast_cancer('samples/real_images.png')

Probabilities for all classes:  [0.02551426738500595, 0.9744857549667358]
Breast cancer detected
Probabilities for all classes:  [0.02551426738500595, 0.9744857549667358]
Breast cancer detected
Probabilities for all classes:  [1.0, 1.9566603892684498e-09]
Breast cancer not detected
Probabilities for all classes:  [1.0, 4.660057496136405e-12]
Breast cancer not detected
Probabilities for all classes:  [1.0, 3.045242280563798e-08]
Breast cancer not detected
Probabilities for all classes:  [1.0, 1.1020956591778486e-08]
Breast cancer not detected
Probabilities for all classes:  [0.9999998807907104, 6.471354652148875e-08]
Breast cancer not detected
Probabilities for all classes:  [0.026084695011377335, 0.9739152789115906]
Breast cancer detected


# Deleting endpoint

In [33]:
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '79763e15-bd29-48a1-ada4-bbc5bed38ee4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '79763e15-bd29-48a1-ada4-bbc5bed38ee4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 07 May 2020 19:32:46 GMT'},
  'RetryAttempts': 0}}